<center><img alt="" src="images/cover_sma.jpg"/></center> 

## <center><font color="blue">Pendahuluan Digital Data Gathering</font><br> <b>(C) Taufik Sutanto - 2020<br>https://tau-data.id/sma-01/

# Outline:

1. Social Media Crawling
2. Social Media Streaming
3. Sekilas Privacy & Ethics Data Gathering
4. Scrapping Media Social & Website
5. Loading Local Documents

In [ ]:
# Hanya jalankan jika menggunakan Google Colab
# Jika menjalankan program ini secara lokal (Anaconda/WinPython) sebaiknya dijalankan di Terminal/Command Prompt

!pip install unidecode twython tweepy beautifulsoup4 tika

In [14]:
import warnings; warnings.simplefilter('ignore')
import tweepy, json, nltk, urllib.request, requests
from urllib.request import Request, urlopen
from twython import TwythonStreamer
from bs4 import BeautifulSoup as bs

# Data Gathering: Crawling, Streaming, Scrapping

<h2 id="Aturan-twitter">Aturan, bentuk data, &amp; error codes twitter</h2>

<ol>
	<li>
	<p><a href="https://dev.twitter.com/rest/public/rate-limiting" target="_blank">https://</a><a href="https://dev.twitter.com/rest/public/rate-limiting" target="_blank">dev.twitter.com/rest/public/rate-limiting</a></p>
	</li>
	<li>
	<p><a href="https://dev.twitter.com/overview/terms/agreement-and-policy" target="_blank">https://dev.twitter.com/overview/terms/agreement-and-policy</a></p>
	</li>
	<li>
	<p><a href="https://dev.twitter.com/overview/api/response-codes" target="_blank">https://</a><a href="https://dev.twitter.com/overview/api/response-codes" target="_blank">dev.twitter.com/overview/api/response-codes</a></p>
	</li>
	<li>
	<p><a href="https://dev.twitter.com/overview/api/tweets" target="_blank">https://</a><a href="https://dev.twitter.com/overview/api/tweets" target="_blank">dev.twitter.com/overview/api/tweets</a></p>
	</li>
</ol>


In [ ]:
# Contoh API Keys (Sesuaikan dengan API keys masing-masing)
Ck = '' # consumer_key
Cs = '' # consumer_secret
At = '-' # access_token
As = '' # access_secret
'Done'

In [15]:
def twitter_connect(Ck, Cs, At, As, verbose = 0):
    try:
        auth = tweepy.OAuthHandler(Ck, Cs)
        auth.set_access_token(At, As)
        twitter = tweepy.API(auth, timeout=120)
        if verbose != 0:
            user = twitter.verify_credentials()
            print('Welcome "%s" you are now connected to twitter server' %user.name)
        return twitter
    except:
        print("Connection failed, please check your API keys or connection")
        return None

In [16]:
# Koneksi ke twitter
twitter = twitter_connect(Ck, Cs, At, As, verbose = 1)

Welcome "Taufik Sutanto" you are now connected to twitter server


In [17]:
# Max 100 tweet per "API call"
topic = 'psbb jakarta'
N = 100 # jumlah tweet yang ingin diambil
bahasa = 'id'
T = twitter.search(q=topic, lang=bahasa, count=N, tweet_mode = 'extended')
tweets = [t._json for t in T]
print(' Berhasil mendapatkan {} tweets'.format(len(tweets)))

 Berhasil mendapatkan 100 tweets


In [19]:
# Datanya berbentuk JSON
tweets[0]['created_at'], tweets[0]['full_text'], 

('Wed Oct 21 00:54:05 +0000 2020',
 'RT @TRANSTV_CORP: Masuk kembali ke masa PSBB Transisi, kini traveler bisa kembali melangsungkan pernikahan di hotel. Hanya wajib lapor dulu…')

In [30]:
tweets[0]

{'created_at': 'Wed Oct 21 00:54:05 +0000 2020',
 'id': 1318717158247989249,
 'id_str': '1318717158247989249',
 'full_text': 'RT @TRANSTV_CORP: Masuk kembali ke masa PSBB Transisi, kini traveler bisa kembali melangsungkan pernikahan di hotel. Hanya wajib lapor dulu…',
 'truncated': False,
 'display_text_range': [0, 140],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'TRANSTV_CORP',
    'name': 'TRANS TV',
    'id': 125225621,
    'id_str': '125225621',
    'indices': [3, 16]}],
  'urls': []},
 'metadata': {'iso_language_code': 'in', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1088628266263441408,
  'id_str': '1088628266263441408',
  'name': 'maher',
  'screen_name': 'maher68554742',
  'locat

In [20]:
# Contoh mengakses data spesifik pada tweet yang pertama:
print('tweet pertama oleh "{}" : "{}"'.format(tweets[0]['user']['screen_name'],tweets[0]['full_text']))

tweet pertama oleh "maher68554742" : "RT @TRANSTV_CORP: Masuk kembali ke masa PSBB Transisi, kini traveler bisa kembali melangsungkan pernikahan di hotel. Hanya wajib lapor dulu…"


# Bagaimana kalau mau mengambil data lebih banyak

In [24]:
# Next data
last_id = T[-1]._json['id'] - 1
last_id
T = twitter.search(q=topic, lang=bahasa, count=N, tweet_mode = 'extended', max_id=last_id)
tweets.extend([t._json for t in T])

print('Jumlah data sekarang = ', len(tweets))

Jumlah data sekarang =  300


# Lalu Loop terus sampai "T" False (kosong/None)

# Menyimpan hasil crawling? (Sebaiknya di NoSQL)

In [25]:
def saveTweets(tweets, file='Tweets.json'): #in Json Format
    with open(file, 'w') as f:
        for t in tweets:
            try:
                f.write(json.dumps(t)+'\n')
            except:
                pass

In [26]:
# Menyimpan hasil crawling twitter
fileName = 'data/tweets_sma-01.json'
saveTweets(tweets,file=fileName)
print('Saved to '+fileName)

Saved to data/tweets_sma-01.json


# Load Kembali?

In [27]:
def loadTweets(file='Tweets.json'):
    f=open(file,encoding='utf-8', errors ='ignore', mode='r')
    T=f.readlines();f.close()
    for i,t in enumerate(T):
        T[i] = json.loads(t.strip())
    return T

In [28]:
# Me-load kembali jika (misal) analisa ingin dilakukan di lain waktu
# Sengaja nama variabelnya saya bedakan (T2)
T2 = loadTweets(file='data/tweets_sma-01.json')
print('tweet pertama oleh "{}" : "{}"'.format(T2[0]['user']['screen_name'],T2[0]['full_text']))

tweet pertama oleh "maher68554742" : "RT @TRANSTV_CORP: Masuk kembali ke masa PSBB Transisi, kini traveler bisa kembali melangsungkan pernikahan di hotel. Hanya wajib lapor dulu…"


In [29]:
# Contoh mengambil hanya data tweet
D = [t['full_text'] for t in T2]
D[:5] # 5 tweet pertama

['RT @TRANSTV_CORP: Masuk kembali ke masa PSBB Transisi, kini traveler bisa kembali melangsungkan pernikahan di hotel. Hanya wajib lapor dulu…',
 'RT @TRANSTV_CORP: Masuk kembali ke masa PSBB Transisi, kini traveler bisa kembali melangsungkan pernikahan di hotel. Hanya wajib lapor dulu…',
 'RT @tempodotco: PSBB Transisi, 77 RPTRA di Jakarta Utara Dibuka Lagi #TempoMetro https://t.co/yipPjc3J0D',
 'Pemprov DKI Jakarta akhirnya mengizinkan bioskop buka saat PSBB transisi. Namun, kebijakan ini masih sulit membangkitkan bisnis bioskop yang telah terpuruk selama pandemii Covid-19. #Analisis\nhttps://t.co/WeK7rk2ieU',
 'Hari Ke-9 PSBB Transisi, Jumlah Positif Corona di Jakarta Tambah 964 Kasus https://t.co/olB2lkHhTb #DariSuara']

# Pemilihan KeyWords yang baik

### https://medium.com/lingvo-masino/how-to-choose-keywords-in-twitter-9c3b85c50290


In [31]:
# Mari kita coba #1
twitter = twitter_connect(Ck, Cs, At, As)
topic = 'from:taudataid'
n = 10 # Contoh jumlah tweet yang ingin diambil
T = twitter.search(q=topic, lang=bahasa, count=N, tweet_mode = 'extended')
tweets = [t._json for t in T]
isiTweet = [t['full_text'] for t in tweets]
isiTweet

['Infographics tentang "Rancangan Undang-undang Perlindungan Data Pribadi" dari AntaraNews.\n\nDraft RUU-PDP:\nhttps://t.co/YWqsBMCMMr\nSumber Gambar: https://t.co/QNvCrO0B1z\nKeterangan lebih lanjut:\nhttps://t.co/KiRLNWP5yA\n\n#DataScientist #Privacy #Ethics #Regulation #GDPR #Indonesia https://t.co/cLxy0qaqrS',
 'EDA-03: Pendahuluan Unsupervised Learning (k-Means)\n\nVideo: https://t.co/nKLzeZMCgk\nCode, Module, &amp; Referensi: https://t.co/ZLxsmnmscq\nForum Diskusi : https://t.co/x7WWv92cH1\n\n#unsupervisedLearning #eda #clustering #python #scikitLearn #kmeans',
 'EDA-02: Visualisasi Dasar\n\nVideo: https://t.co/ioUFoyzi8I\nModule &amp; Code: https://t.co/Gg4ghqXyHf\nForum Diskusi/Tanya-jawab: https://t.co/x7WWv92cH1\n\n#eda #visualisasi #python #datascience #pandas #seaborn #plt #numpy #folium',
 '* Tahukah anda bahwa kita tidak selalu boleh melakukan imputasi pada missing values?\n* Apa ya bedanya outlier dan noise?\n* Ternyata tidak bijak untuk meng-"exclude" outlier, kenapa ya

## Go to: https://www.latlong.net/convert-address-to-lat-long.html

## Get Latitude dan Longitude dari alamat yang diinginkan

**Catt**: Proses ini bisa juga digantikan dengan Google Maps API (gratis 40.000 query/bulan) dan bisa dijalankan langsung dalam program Python

In [32]:
# Mari kita coba #3 gunakan google (map) untuk koordinat suatu lokasi
# http://thoughtfaucet.com/search-twitter-by-location/
# misal search tweet tentang "makanan" di Depok dan sekitarnya

auth = tweepy.auth.OAuthHandler(Ck, Cs)
auth.set_access_token(At, As)
api = tweepy.API(auth)

Geo, N = "-6.402484,106.794243,30km", 5  # HAti-hati jangan ada spasi di Lat-Lon-Radius
qry = 'makanan'
for tweet in tweepy.Cursor(api.search, q=qry, count=100, geocode=Geo).items(N):
    print([tweet.created_at, tweet.text.encode('utf-8'), tweet.user.id, tweet.geo])

[datetime.datetime(2020, 10, 21, 1, 14, 1), b'RT @VIVAcoid: Dukung Mahasiswa, Warga Bagikan Makanan di Kawasan Patung Kuda https://t.co/VY2McaGae1 #nasional', 776017308, None]
[datetime.datetime(2020, 10, 21, 1, 9, 29), b'RT @bankmandiri: Sebutkan nama makanan khas Indonesia dalam gambar.\n\n#MandirikanKaryaNegeri #BanggaBuatanIndonesia https://t.co/JePkdHoKaj', 280894154, None]
[datetime.datetime(2020, 10, 21, 1, 8, 59), b'20. Dokter ini bilang gak ada makanan yang dipantang. Wow aku terkejoet dan sangat bahagia lah dengernya bebas bisa\xe2\x80\xa6 https://t.co/MCJfbQ1cek', 1189188980, None]
[datetime.datetime(2020, 10, 21, 1, 8, 52), b'7. Ada sekitar 3 sampe 4 x ketemu dokternya. Waktu konsul sama dokter ini bener bener dikasih pantangan makanan ban\xe2\x80\xa6 https://t.co/pb1j2k9RWP', 1189188980, None]
[datetime.datetime(2020, 10, 21, 1, 8, 33), b'sarapan dulu biar kuat menghadapi kenyataan\xf0\x9f\xa4\xa3\xf0\x9f\xa4\xa3\n\nmakanan ini mewakili menu makan ara hari ini karna dibiki

In [33]:
# Streaming tweets. Untuk percobaan pilih topicS sesuatu yg sedang trending/populer "saat ini".
# Atau bisa coba dengan mengirim tweet sendiri :)
def streamTwitter(topicS, lang):
    class MyStreamer(TwythonStreamer):
        def on_success(self, data):
            global count
            count+=1
            print('tweet from {}, post: {}'.format(data['user']['screen_name'], data['text']))
            if count==maxTweet:
                print('\nFinished streaming %.0f tweets' %(maxTweet)); self.disconnect()
        def on_error(self, status_code, data):
            print('Error Status = %s' %status_code); self.disconnect()

    while count<maxTweet:
        stream = MyStreamer(Ck, Cs, At, As)
        stream.statuses.filter(track=topicS)

In [34]:
maxTweet, count = 2, 0 # Rubah sesuai dengan kebutuhan, Untuk percobaan ini cukup (misal) 3 tweet
lang = set(['en','id']) # bahasa bisa dipilih > 1
topicS = ['taudata'] # Bisa>1

streamTwitter(topicS, lang)

tweet from dindinproject, post: Taudata
tweet from han_farhan8299, post: Pagi pagi lagi belajar sama taudata tentang SMA

Finished streaming 2 tweets


# Data Gathering Ethics-Regulation

* RUU PDP + ITE
* ToS
* Robots.txt
* Consent

### https://tau-data.id/scraping/

# Scrapping 

1. Go to: https://twitter.com/search-advanced 
2. Search sesuai keinginan/kebutuhan
3. Copy URL
4. Ikuti instruksi dibawah

### Silahkan Loop dan Modifikasi seperlunya, tapi hormati robots.txt twitter

In [35]:
# Server tidak suka request tanpa "header"/identifier

h = {'User-Agent' : "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.27 Safari/537.17"}
url = 'https://twitter.com/search?q=%22janda%20bolong%22%20until%3A2020-10-21%20since%3A2019-01-01&src=typed_query'

req = Request(url, headers = h)
resp = urlopen(req).read()
soup = bs(resp,'html.parser')
Tweets = soup.find_all(class_= 'tweet-text')
Tweets = [bs(str(t), "lxml").text for t in Tweets]
print(Tweets[0], Tweets[1], sep='\n') # Print 2 pertama


  "Bubbles, Fads, and Price Volatilty: The Janda Bolong Market Crash of 2020" twitter.com/ujangw/status/…



  A formula is an expression which calculates the value of a cell. Functions are predefined formulas and are already available in #Excel: excel-easy.com/introduction/f… pic.twitter.com/bBPYSf0ZFu




# Web Scrapping?

In [36]:
import urllib.request
from bs4 import BeautifulSoup as bs

URL = 'https://fst.uinjkt.ac.id/'
Doc = urllib.request.urlopen(URL).read()
Doc = bs(Doc,'lxml').text
print(Doc)








Fakultas Sains dan Teknologi – Portal Resmi Fakultas Sains dan Teknologi





 

























Skip to content


  














Search for:



 

Prodi Matematika
Agribisnis
Magister Agribisnis
Biologi
Kimia
Fisika
Sistem Informasi
Teknik Informatika
Teknik Pertambangan
Pusat Laboratorium Terpadu
 








AKREDITASI

Borang Fakultas
Magister Agribisnis

MA – Standar 1
MA – Standar 2
MA – Standar 3
MA – Standar 4
MA – Standar 5
MA – Standar 6
MA – Standar 7




AUN QA

NEWS
SAR DOC
APPENDIX


LAKIP FST

LAKIP FST 2017
LAKIP FST 2018


KIMIA

SK Borang Prodi Kimia


 






Search for:













   


Dekanat




 


DEMA FST / Jadwal Dekan / Kegiatan Fakultas / Pojok Dosen / Pojok Mahasiswa / SEMA FST PBAK FAKULTAS SAINS DAN TEKNOLOGI 2020
11 Sep, 2020 




 


Agribisnis / Jadwal Dekan / Kegiatan Fakultas Praktek Kerja Lapangan (PKL)
9 Sep, 2020 





 



BERANDA
PROFIL

Pimpinan dan Staff FST UIN Jakarta
Dosen Pengajar FST
Program Studi

Prodi Matematika
Prodi 

# Loading Local Documents

1. Instalasi Java
 - JDK 8 ... ingat harus JDK 8
 - https://www.oracle.com/id/java/technologies/javase/javase-jdk8-downloads.html
 - Set Java Home Directory
2. Instalasi Tika Server :
 - Download Tika App Executable Java Jar: https://www.apache.org/dyn/closer.cgi/tika/tika-app-1.24.1.jar
 - Put in Python home directory
3. Instalasi Module Python Tika
4. Reading pdf Files (Python Code below)

### Tika can read Pdf, DocX, PPTX, etc.

In [39]:
from tika import parser#, unpack

def readDocs(file):
    if 'pdf' in file:
        headers = {'X-Tika-PDFextractInlineImages': 'true',} 
        raw = parser.from_file(file, headers=headers)
    else:
        raw = parser.from_file(file)
    if 'content' in raw.keys():
        return raw['content']
    else:
        return None

In [40]:
try:
    doc = readDocs('data/contoh.pdf')
except:
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/contoh.pdf
    doc = readDocs('data/contoh.pdf')
    
print(doc)

















































BUTIR-BUTIR PEDOMAN PENGHAYATAN DAN PENGAMALAN PANCASILA 
 
   
Lima asas dalam Pancasila dijabarkan menjadi 36 butir pengamalan, sebagai pedoman praktis bagi 
pelaksanaan Pancasila. 
 
Butir-butir Pancasila ditetapkan dalam Ketetapan MPR No. II/MPR/1978 tentang Ekaprasetia 
Pancakarsa. 
 
I. SILA PERTAMA : KETUHANAN YANG MAHA ESA 

1. Percaya dan Takwa kepada Tuhan Yang Maha Esa sesuai dengan agama dan kepercayaan 
masing-masing menurut dasar kemanusiaan yang adil dan beradab. 

2. Hormat menghormati dan bekerjasama antar pemeluk agama & penganut-penganut 
kepercayaan yang berbeda-beda sehingga terbina kerukunan hidup. 

3. Saling hormat-menghormati kebebasan menjalankan ibadah sesuai dengan agama dan 
kepercayaannya. 

4. Tidak memaksakan suatu agama dan kepercayaan kepada orang lain. 
 
II. SILA KEDUA : KEMANUSIAAN YANG ADIL DAN BERADAB 

1. Mengakui persamaan derajat, persamaan hak dan persamaan kewajiban antara sesama 
manusia. 

2 S

# <center><font color="blue"> End of Module

<hr />
